In [2]:
import pandas as pd
import joblib
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score

# Đọc dữ liệu
def load_data(train_path, test_path):
    train_dt = pd.read_csv(train_path)
    test_dt = pd.read_csv(test_path)
    return train_dt, test_dt

train_dt, test_dt = load_data('cityu10c_train_dataset.csv', 'cityu10c_test_dataset.csv')

# Hiển thị thông tin dữ liệu
print(train_dt.info())
print(train_dt.describe())
print(test_dt.info())
print(test_dt.describe())

# Xử lý dữ liệu thiếu và loại bỏ cột không cần thiết
def preprocess_missing_values(df):
    df = df.copy()
    df.drop(columns=['ApplicationDate'], inplace=True, errors='ignore')
    return df

# Lấy danh sách các cột phân loại có ít giá trị duy nhất
def get_categorical_candidates(df, threshold=15):
    unique_counts = df.nunique()
    categorical_candidates = unique_counts[unique_counts < threshold].index.tolist()
    return [col for col in categorical_candidates if col != 'ID']

# Xử lý dữ liệu
train_dt = preprocess_missing_values(train_dt)
train_target = train_dt.pop('LoanApproved')
test_dt = preprocess_missing_values(test_dt)

categorical_candidates = get_categorical_candidates(train_dt)
numerical_candidates = [col for col in train_dt.columns if col not in categorical_candidates]

# Transformers
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_candidates),
        ('cat', categorical_transformer, categorical_candidates)
    ]
)

# Tạo pipeline với LightGBM
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', lgb.LGBMClassifier())
])

# Tách tập train và validation
X_train, X_val, y_train, y_val = train_test_split(train_dt, train_target, test_size=0.2, random_state=42)

# Thiết lập tham số cho GridSearch
param_grid = {
    'classifier__num_leaves': [31],
    'classifier__min_data_in_leaf': [5],
    'classifier__learning_rate': [0.12],
    'classifier__n_estimators': [500],
    'classifier__max_depth': [-1]
}

# Tối ưu mô hình
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='f1', n_jobs=-1, verbose=1, refit=True)
grid_search.fit(X_train, y_train)

# Lấy mô hình tốt nhất
best_model = grid_search.best_estimator_
print("Best parameters found:", grid_search.best_params_)

# Dự đoán trên tập validation
y_pred = best_model.predict(X_val)
f1 = f1_score(y_val, y_pred)
print(f'Optimized F1 Score: {f1:.4f}')

# Dự đoán trên tập test
test_predictions = best_model.predict(test_dt)

# Lưu mô hình
if isinstance(best_model, Pipeline):
    joblib.dump(best_model, 'trained_pipeline.pkl')
    print("🎉 Pipeline đã được lưu thành công vào trained_pipeline.pkl")
else:
    print("❌ LỖI: best_model không phải là pipeline!")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          8000 non-null   int64  
 1   ApplicationDate             8000 non-null   object 
 2   Age                         8000 non-null   int64  
 3   AnnualIncome                8000 non-null   int64  
 4   CreditScore                 8000 non-null   int64  
 5   EmploymentStatus            8000 non-null   object 
 6   EducationLevel              8000 non-null   object 
 7   Experience                  8000 non-null   int64  
 8   LoanAmount                  8000 non-null   int64  
 9   LoanDuration                8000 non-null   int64  
 10  MaritalStatus               8000 non-null   object 
 11  NumberOfDependents          8000 non-null   int64  
 12  HomeOwnershipStatus         8000 non-null   object 
 13  MonthlyDebtPayments         8000 

c:\Apps\miniconda3forpython\envs\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Apps\miniconda3forpython\envs\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
